In [22]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder 
import tinysegmenter

In [2]:
devdata_file = pd.ExcelFile("devdata.xlsx")
devdata = pd.read_excel(devdata_file)

In [3]:
# replace nan with '' in ACCOUNT_NAME
devdata.ACCOUNT_NAME.fillna('', inplace=True)

In [4]:
segmenter = tinysegmenter.TinySegmenter()
vectorizer_ng3 = CountVectorizer(ngram_range=(1,3),tokenizer=segmenter.tokenize)

In [5]:
all_transform_matrix = vectorizer_ng3.fit_transform(devdata['ACCOUNT_NAME'])

In [6]:
all_transform_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
# Convert bow_matrix into a DataFrame
all_transform_df = pd.DataFrame(all_transform_matrix.toarray())

In [8]:
# Map the column names to vocabulary 
all_transform_df.columns = vectorizer_ng3.get_feature_names()

In [20]:
all_other_df = pd.get_dummies(devdata[['ACCOUNTING_SYSTEM']])

In [21]:
all_other_df

,ACCOUNTING_SYSTEM_Biz∫,ACCOUNTING_SYSTEM_Company,ACCOUNTING_SYSTEM_EXPLANNER,ACCOUNTING_SYSTEM_FutureStage,ACCOUNTING_SYSTEM_GLOVIA,ACCOUNTING_SYSTEM_MINCS,ACCOUNTING_SYSTEM_OBIC,ACCOUNTING_SYSTEM_OBIC7,ACCOUNTING_SYSTEM_Oracle EBS,ACCOUNTING_SYSTEM_OracleEBS,ACCOUNTING_SYSTEM_SAP,ACCOUNTING_SYSTEM_SUN,ACCOUNTING_SYSTEM_SUPER STREAM CORE,ACCOUNTING_SYSTEM_SuperStream,ACCOUNTING_SYSTEM_SuperStreamNX,ACCOUNTING_SYSTEM_SuperStreem,ACCOUNTING_SYSTEM_TASK,ACCOUNTING_SYSTEM_国内共通会計システム（SAP),ACCOUNTING_SYSTEM_経理システム（自社開発）
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
le = LabelEncoder()

# all_other_df['COMPANY'] = le.fit_transform(all_other_df['COMPANY'])
all_other_df['ACCOUNTING_SYSTEM'] = le.fit_transform(all_other_df['ACCOUNTING_SYSTEM'])
#all_other_df['BP_DIV'] = le.fit_transform(all_other_df['BP_DIV'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [11]:
all_contat = pd.concat([all_other_df.reset_index(drop=True), all_transform_df], axis=1)

In [12]:
full_contat = pd.concat([all_contat.reset_index(drop=True), devdata['ACCOUNT_GROUP']], axis=1)

In [13]:
full_train_contat, full_test_contat = train_test_split(full_contat, test_size = 0.2, random_state=200)

# Bag of Words (BOW) modeling

In [14]:
clf = MultinomialNB()

In [15]:
y_train = full_train_contat['ACCOUNT_GROUP'].values
y_test = full_test_contat['ACCOUNT_GROUP'].values

In [16]:
# Fit the classifier
clf.fit(full_train_contat.drop(['ACCOUNT_GROUP'], axis = 1), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
# Measure the accuracy
accuracy = clf.score(full_test_contat.drop(['ACCOUNT_GROUP'], axis = 1), y_test)
print("The accuracy of the classifier on the test set is %.3f" % accuracy)


The accuracy of the classifier on the test set is 0.576
